<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_Threads_Java_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Java**

//TODO: Formatear y agregar documentacion

In [ ]:
%%writefile Tp1Parte2.java
import java.util.ArrayList;
import java.util.List;
import java.util.Random;

public class Tp1Parte2
{
  private final static int LOWER_LIMIT_RANDOM = -32;
  private final static int HIGHER_LIMIT_RANDOM = 32;

  public static void main(String[] args)
  {
    if (args.length < 1)
    {
      System.out.println("Error. Cantidad incorrecta de argumentos");
      return;
    }
    try
    {
      int matrixSize = Integer.parseInt(args[args.length - 1]);

      IntegerMatrix matrixA = new IntegerMatrix(matrixSize);
      matrixA.randomizeData(LOWER_LIMIT_RANDOM, HIGHER_LIMIT_RANDOM);

      IntegerMatrix matrixB = new IntegerMatrix(matrixSize);
      matrixB.randomizeData(LOWER_LIMIT_RANDOM, HIGHER_LIMIT_RANDOM);

      IntegerMatrix matrixCS = IntegerMatrix.multiply(matrixA, matrixB, new SecuencialMultiplicationStrategy());
      IntegerMatrix matrixCH = IntegerMatrix.multiply(matrixA, matrixB, new ConcurrentMultiplicationStrategy());

      printResults(matrixA, matrixB, matrixCS, matrixCH);
    } catch (IllegalArgumentException illegalArgumentException)
    {
      System.out.println("No se indico un tamaño correcto de matriz");
    } catch (Exception e)
    {
      System.out.println(e.getMessage());
    }
  }

  private static void printResults(IntegerMatrix m1, IntegerMatrix m2, IntegerMatrix result, IntegerMatrix result2)
  {
    System.out.println("---- Matriz A ----");
    printMatrix(m1.get());
    System.out.println("---- Matriz B ----");
    printMatrix(m2.get());
    System.out.println("---- Matriz CS ----");
    printMatrix(result.get());
    System.out.println("---- Matriz CH ----");
    printMatrix(result2.get());
    boolean areEqual = result.equals(result2);
    System.out.printf("Las matrices son %s%n", areEqual ? "iguales" : "distintas");
  }

  public static void printMatrix(int[][] matrix)
  {
    for (int[] row : matrix)
    {
      for (int element : row)
      {
        System.out.print("| " + String.format("%7s", element) + " ");
      }
      System.out.println("|");
    }
    System.out.println();
  }

  public static class ConcurrentMultiplicationStrategy implements MultiplicationStrategy
  {
    @Override
    public IntegerMatrix multiply(IntegerMatrix matrixA, IntegerMatrix matrixB, IntegerMatrix resultMatrix)
        throws Exception
    {
      int iterations = matrixA.rows();
      List<Thread> threadsExecuting = generateThreads(matrixA, matrixB, iterations, resultMatrix);
      waitThreads(threadsExecuting);
      return resultMatrix;
    }

    private void waitThreads(List<Thread> threadsExecuting) throws InterruptedException
    {
      for (Thread thread : threadsExecuting)
      {
        thread.join();
      }
    }

    private List<Thread> generateThreads(IntegerMatrix matrixA, IntegerMatrix matrixB, int iterations, IntegerMatrix resultMatrix)
    {
      List<Thread> threadsExecuting = new ArrayList<>();
      for (int i = 0; i < iterations; i++)
      {
        final int[] row = matrixA.get(i);
        final int rowIndex = i;
        Thread thread = new Thread(() -> multiplyRow(row, matrixB, resultMatrix, rowIndex));
        thread.start();
        threadsExecuting.add(thread);
      }
      return threadsExecuting;
    }


    private void multiplyRow(int[] row, IntegerMatrix matrixB, IntegerMatrix resultMatrix, int rowIndex)
    {
      for (int i = 0; i < matrixB.columns(); i++)
      {
        for (int j = 0; j < row.length; j++)
        {
          int currentValue = resultMatrix.get(rowIndex, i);
          int updatedValue = currentValue + row[j] * matrixB.get(j, i);
          resultMatrix.set(rowIndex, i, updatedValue);
        }
      }
    }
  }

  public static class IntegerMatrix
  {
    private final int[][] data;
    private final int rows;
    private final int columns;

    /**
     * @param size the size used for rows and columns of the matrix
     * @throws IllegalArgumentException if {@code rows} or {@code columns} are less than or equal to zero.
     */
    public IntegerMatrix(int size) throws IllegalArgumentException
    {
      this(size, size);
    }

    /**
     * @param rows    the row dimension of the matrix
     * @param columns the column dimension of the matrix
     * @throws IllegalArgumentException if {@code rows} or {@code columns} are less than or equal to zero.
     */
    public IntegerMatrix(int rows, int columns) throws IllegalArgumentException
    {
      if (rows <= 0 || columns <= 0)
      {
        throw new IllegalArgumentException("Debe especificar un numero entero positivo para las dimensiones de la matriz");
      }
      this.rows = rows;
      this.columns = columns;
      data = new int[rows][columns];
    }

    public final int rows()
    {
      return rows;
    }

    public final int columns()
    {
      return columns;
    }

    /**
     * Obtiene el elemento almacenado en la posicion indicada.
     *
     * @param row    el indice basado en cero de la fila deseada.
     * @param column el indice basado en cero de la columna deseada.
     */
    public final int get(int row, int column) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || column >= columns())
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      return data[row][column];
    }

    /**
     * Obtiene el array que conforma la fila indicada.
     *
     * @param row el indice basado en cero de la fila deseada.
     */
    public final int[] get(int row) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows())
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      return data[row];
    }

    public final int[][] get()
    {
      return data;
    }

    public final void set(int row, int column, Integer value) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || row < 0 || column >= columns() || column < 0)
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      data[row][column] = value;
    }

    public final void set(int row, int[] value) throws ArrayIndexOutOfBoundsException
    {
      if (row >= rows() || row < 0)
      {
        throw new ArrayIndexOutOfBoundsException();
      }
      data[row] = value;
    }

    public static IntegerMatrix multiply(IntegerMatrix matrixA, IntegerMatrix matrixB, MultiplicationStrategy multiplicationStrategy) throws Exception
    {
      if (!matrixA.isCompatibleForMultiplication(matrixB))
      {
        throw new IllegalArgumentException("Las matrices no son compatibles para su multiplicacion.");
      }
      IntegerMatrix result = new IntegerMatrix(matrixA.columns(), matrixB.rows());
      return multiplicationStrategy.multiply(matrixA, matrixB, result);
    }

    private boolean isCompatibleForMultiplication(IntegerMatrix matrixB)
    {
      return this.columns() == matrixB.rows();
    }

    public void randomizeData(int minValue, int maxValue)
    {
      for (int i = 0; i < rows(); i++)
      {
        for (int j = 0; j < columns(); j++)
        {
          data[i][j] = RangeRandom.randomInt(minValue, maxValue);
        }
      }
    }

    @Override
    public boolean equals(Object other)
    {
      // If the object is compared with itself then return true
      if (other == this)
      {
        return true;
      }

      /*Check if o is an instance of Complex or not "null instanceof [type]" also returns false */

      if (this.getClass()!=other.getClass())
      {
        return false;
      }
      IntegerMatrix otherMatrix = (IntegerMatrix) other;

      // typecast o to Complex so that we can compare data members

      if (this.rows() != otherMatrix.rows() || this.columns() != otherMatrix.columns())
      {
        return false;
      }

      for (int i = 0; i < rows(); i++)
      {
        for (int j = 0; j < columns(); j++)
        {
          if (this.get(i, j) != otherMatrix.get(i, j))
          {
            return false;
          }
        }
      }
      return true;
    }
  }

  public interface MultiplicationStrategy
  {
    IntegerMatrix multiply(IntegerMatrix matrixA, IntegerMatrix matrixB, IntegerMatrix resultMatrix) throws Exception;
  }

  public static class RangeRandom
  {
    public static final Random random = new Random(System.currentTimeMillis());

    public static int randomInt(int start, int end)
    {
      return random.nextInt((end - start) + 1) + start;
    }
  }

  public static class SecuencialMultiplicationStrategy implements MultiplicationStrategy
  {
    @Override
    public IntegerMatrix multiply(IntegerMatrix matrixA, IntegerMatrix matrixB, IntegerMatrix resultMatrix)
    {
      for (int i = 0; i < matrixA.rows(); ++i)
      {
        for (int j = 0; j < matrixB.columns(); ++j)
        {
          for (int k = 0; k < matrixB.rows(); ++k)
          {
            resultMatrix.set(i, j, resultMatrix.get(i, j) + matrixA.get(i, k) * matrixB.get(k, j));
          }
        }
      }
      return resultMatrix;
    }
  }
}

Overwriting Tp1Parte2.java


In [ ]:
!java Tp1Parte2.java "2"

---- Matriz A ----
|       9 |      32 |
|      21 |      19 |

---- Matriz B ----
|       8 |      21 |
|     -21 |     -28 |

---- Matriz CS ----
|    -600 |    -707 |
|    -231 |     -91 |

---- Matriz CH ----
|    -600 |    -707 |
|    -231 |     -91 |

Las matrices son iguales


In [ ]:
!java --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
